# lilac

Lilac helps you curate data for LLMs, from RAGs to fine-tuning datasets.

https://github.com/lilacai/lilac

https://www.lilacml.com/

In [ ]:
!pip3 install --upgrade pip

In [ ]:
pip install lilac #[all]

In [1]:
import os

gradio_server_port = int(os.environ.get('GRADIO_PORT'))
gradio_root_path = os.environ.get('GRADIO_BASE_URL')

print(f"This port used by UI apps in the container: {gradio_server_port}")
print(f"Will be exposed at the following url on the virtual machine: {gradio_root_path}")

This port used by UI apps in the container: 7860
Will be exposed at the following url on the virtual machine: /notebooks/gradio


In [2]:
from IPython.display import display, HTML

display(HTML(f'<a href="http://192.168.1.24{gradio_root_path}/" target="_blank">Click here to access the UI app</a>'))

In [4]:
import lilac as ll

project_dir = '~/bank_project'
server = ll.start_server(host='0.0.0.0', port=gradio_server_port, project_dir=project_dir)

INFO:     Started server process [31981]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:7860 (Press CTRL+C to quit)


In [5]:
server.stop()

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [31981]
